In [ ]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import cv2
import pandas as pd

In [ ]:
# load the face detector cascade and smile detector CNN
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = load_model('lenet.hdf5')
camera = cv2.VideoCapture('P3.avi')
labels = []
index = []

while True:
    # grab the current frame
    curTimeMiliSec = camera.get(0) #current time
    (grabbed, frame) = camera.read()

    # if we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
    if not grabbed:
        break

    if (curTimeMiliSec % 1000 != 0): continue

    index.append(curTimeMiliSec)

    # reszie the frame, convert it to grayscale, then clone the
    # original frame so we can draw on it later in the program
    frame = imutils.resize(frame, width = 300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frameClone = frame.copy()

    # detect faces in the input frame, then clone the frame so
    # that we can draw on it
    rects = detector.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5,
        minSize = (30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
    
    label = 'NaN'
    for (fX, fY, fW, fH) in rects:
        # extract the ROI of the face from the grayscale image,
        # resize it to a fixed 28x28 pixels, and then prepare the
        # ROI for classification via CNN
        roi = gray[fY: fY + fH, fX: fX + fW]
        roi = cv2.resize(roi, (28, 28))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis = 0)

        # determine the probabilities of both "smiling" and "not similing"
        # then set the label accordingly
        (notSmiling, smiling) = model.predict(roi)[0]
        label = "Smiling" if smiling > notSmiling else "Not Smiling"
    labels.append(label)


    # if 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

In [ ]:
smile_features = pd.Series(labels, index=index)

In [ ]:
pd.DataFrame(smile_features).groupby(0).size()

0
NaN             91
Not Smiling    331
Smiling         52
dtype: int64